# Prestage data for computation

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of config failed: Traceback (most recent call last):
  File "/Users/mclong/miniconda3/envs/so-co2/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/mclong/miniconda3/envs/so-co2/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/mclong/miniconda3/envs/so-co2/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/mclong/miniconda3/envs/so-co2/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_

In [14]:
import os
from subprocess import Popen, PIPE
import tarfile

import xarray as xr
xr.set_options(display_style='text')

import config

## Print storage locations

In [15]:
config.get("project_tmpdir")

error in config: cannot mkdir /glade/work/mclong/so-co2-airborne-obs
setting project_tmpdir to /Users/mclong/scratch/tmp/project_tmpdir


'/Users/mclong/scratch/tmp/so-co2-airborne-obs/project_tmpdir'

In [16]:
config.get("project_tmpdir_obs")

error in config: cannot mkdir /glade/work/mclong/so-co2-airborne-obs/obs-data
setting project_tmpdir_obs to /Users/mclong/scratch/tmp/project_tmpdir_obs


'/Users/mclong/scratch/tmp/so-co2-airborne-obs/project_tmpdir_obs'

In [17]:
config.get("model_data_dir_root")

error in config: cannot mkdir /glade/work/mclong/so-co2-airborne-obs/model-data
setting model_data_dir_root to /Users/mclong/scratch/tmp/model_data_dir_root


'/Users/mclong/scratch/tmp/so-co2-airborne-obs/model_data_dir_root'

In [32]:
config.get("model_data_dir")

'/glade/work/mclong/so-co2-airborne-obs/model-data/Long-etal-2021-SO-CO2-Science'

In [19]:
config.get("dash_asset_fname")

'Long-etal-2021-SO-CO2-Science.tar.gz'

## Get data from DASH repo

In [20]:
if not os.path.isdir(config.get("model_data_dir")):
    # run wget to stage data
    # TODO: support curl too
    cwd = os.getcwd()
    script = f'{cwd}/wget-dash-archive.sh'

    os.chdir(config.get("model_data_dir_root"))

    p = Popen(['bash', script], stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    if p.returncode:    
        print(stderr.decode('UTF-8'))
        print(stdout.decode('UTF-8'))
        raise OSError('data transfer failed')    

    # untar archive
    assert os.path.isfile(config.get("dash_asset_fname")), f'missing {config.get("dash_asset_fname")}'
    tar = tarfile.open(config.get("dash_asset_fname"), "r:gz")
    tar.extractall()
    tar.close()

    os.chdir(cwd)

os.listdir(config.get("model_data_dir"))

error in config: cannot mkdir /glade/work/mclong/so-co2-airborne-obs/model-data
setting model_data_dir_root to /Users/mclong/scratch/tmp/model_data_dir_root
/Users/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/wget-dash-archive.sh: line 4: wget: command not found
/Users/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/wget-dash-archive.sh: line 153: wget: command not found
/Users/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/wget-dash-archive.sh: line 153: wget: command not found
/Users/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/wget-dash-archive.sh: line 211: tac: command not found
/Users/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/wget-dash-archive.sh: line 211: tac: command not found

Running wget-dash-archive.sh version: 
README.md ...Downloading
download failed
Long-etal-2021-SO-CO2-Science.tar.gz ...Downloading
download failed



OSError: data transfer failed

## Check on `intake` datasets

In [34]:
import models

error in config: cannot mkdir /glade/work/mclong/so-co2-airborne-obs
setting project_tmpdir to /Users/mclong/scratch/tmp/project_tmpdir


In [35]:
ds = models.dataset_som_ffn.open_dataset()
ds

<xarray.Dataset>
Dimensions:             (time: 432, lat: 180, lon: 360, d2: 2, bnds: 2)
Coordinates:
  * time                (time) datetime64[ns] 1982-01-15T12:00:00 ... 2017-12...
  * lat                 (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Dimensions without coordinates: d2, bnds
Data variables: (12/15)
    spco2_raw           (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    SFCO2_OCN           (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    spco2_smoothed      (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    SFCO2_OCN_smoothed  (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    sol                 (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    kw                  (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    ...                  ...
    dco2_smoothed       (time, lat, lon) float32 nan nan nan nan ... nan nan nan
    seamask             (lat, lon) int32 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    time_bnds           (time, d2) datetime64[ns] 1981-12-31 ... 2017-12-31
    lat_bnds            (lat, bnds) float32 -90.0 -89.0 -89.0 ... 89.0 89.0 90.0
    lon_bnds            (lon, bnds) float32 -180.0 -179.0 -179.0 ... 179.0 180.0
    area                (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08
Attributes:
    institution:    MPI-MET, Hamburg, Germany (former: ETH Zurich, Switzerland)
    institude_id:   MPI
    model_id:       SOM-FFN
    run_id:         v2018
    contact:        Peter Landschutzer (peter.landschuetzer@mpimet.mpg.de)
    creation_date:  2019-03-21